In [34]:
%load_ext autoreload
%autoreload 2

import sys, os, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb
import shutil

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

In [36]:
results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
print(results_df_old.shape)
print(results_df_old.columns)
results_df_old.head()

(243, 41)
Index(['lr', 'net', 'bias', 'name', 'seed', 'tags', 'test', 'n_val', 'train',
       'y_low', 'device', 'logger', 'n_test', 'run_id', 'y_high', 'log_dir',
       'n_train', 'predict', 'version', 'data_dir', 'eps_mult', 'n_splits',
       'criterion', 'do_return', 'earlystop', 'grid_size', 'noise_std',
       'only_flip', 'batch_size', 'max_epochs', 'data_kwargs', 'num_workers',
       'generate_data', 'model_summary', 'args_processed', 'do_return_model',
       'transform_kwargs', 'persistent_workers', 'run_name', 'test_loss',
       'space_length'],
      dtype='object')


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,num_workers,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss,space_length
0,0.001,TrainP,False,None,3,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,lucky-lamp-537,0.000743,NaN
1,0.001,TrainP,False,None,2,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,glistening-dog-536,0.000789,NaN
2,0.001,TrainP,False,None,1,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,brilliant-orchid-535,0.000772,NaN
3,0.010,TrainP,False,None,3,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,fortuitous-paper-534,0.001858,NaN
4,0.010,TrainP,False,None,2,[sine2d-rot],True,1000,True,1.0,...,18,False,False,True,False,"{'eps_mult': [0, 1, 1, 1], 'only_flip': False}",True,bright-lamp-533,0.001986,NaN


In [37]:
api = wandb.Api()
runs = api.runs('eliasdubbeldam/symlie')
run_id_api = {run.id : {'tags':run.tags, 'name':run.name} for run in runs}
print(len(run_id_api))
run_id_api

292


{'g7rsqk3o': {'tags': ['sine2d-rot-predict'], 'name': 'virtuous-fuse-614'},
 'p42eqfqx': {'tags': ['sine2d-rot-predict'], 'name': 'sparkling-paper-613'},
 'dgv8lctw': {'tags': ['sine2d-rot-predict'], 'name': 'crimson-peony-611'},
 't0zssfmp': {'tags': ['sine2d-rot-predict'], 'name': 'glowing-dumpling-612'},
 '16rnzql6': {'tags': ['sine2d-rot-predict'], 'name': 'red-springroll-610'},
 'tn4klaq9': {'tags': ['sine2d-rot-predict'], 'name': 'twinkling-lamp-609'},
 '75zureem': {'tags': ['sine2d-rot-predict'], 'name': 'alight-dragon-607'},
 'x6hffjvm': {'tags': ['sine2d-rot-predict'],
  'name': 'scintillating-cake-607'},
 'fzwk6ncf': {'tags': ['sine2d-rot-predict'], 'name': 'vibrant-dog-606'},
 'ui2tmheh': {'tags': ['sine2d-rot-predict'],
  'name': 'glistening-dumpling-605'},
 'e9k754hw': {'tags': ['sine2d-rot-predict'], 'name': 'dancing-monkey-604'},
 'jtbqan91': {'tags': ['sine2d-rot-predict'], 'name': 'alight-dog-603'},
 'lhnoneb6': {'tags': ['sine2d-rot-predict'], 'name': 'sparkling-lante

In [38]:
REMOVE_RUNS = False

tidy_dirs = {
    'P': {
        'dir'     : os.path.join(store_dir, 'P'), 
        'listdir' : lambda dir: [(dir, dir.split('.')[0]) for dir in os.listdir(dir)],
        'rmdir'   : lambda dir: os.remove(dir),
    },
    'symlie' : {
        'dir'     : os.path.join(log_dir, 'symlie'), 
        'listdir' : lambda dir: [(dir, dir) for dir in os.listdir(dir)],
        'rmdir'   : lambda dir: shutil.rmtree(dir),
    },
    'wandb' : {
        'dir'     : os.path.join(wandb_dir), 
        'listdir' : lambda dir: [(dir, dir.split('-')[-1]) for dir in os.listdir(dir) if dir.startswith('run')],
        'rmdir'   : lambda dir: shutil.rmtree(dir),
    },
}

for tidy_dir, t in tidy_dirs.items():
    count = 0
    for dir_full, dir_strip in t['listdir'](t['dir']):
        if dir_strip not in run_id_api.keys():
            # print(f'Deleting {dir_full}')
            count += 1
            if REMOVE_RUNS:
                t['rmdir'](os.path.join(t['dir'], dir_full))
    print(f'{count} directories removed from {tidy_dir}')

0 directories removed from P
32 directories removed from symlie
32 directories removed from wandb


In [39]:
def exceptions(runs):
    runs_new = []
    for run in runs:

        if (run.state != 'finished'): continue
        if run.id in results_df_old['run_id'].values: continue 
        if 'dev' in run.tags: continue
        
        print(run.id)
        runs_new.append(run)
    return runs_new

runs_selected = exceptions(runs)
print(len(runs_selected))

0


In [40]:
config_list = []

pbar = tqdm(runs_selected)
for run in pbar:
    id = run.id
    pbar.set_description(f'Retreiving wandb {str(id)}')

    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Test loss
    try:
        test_loss_history = run.history(keys=['test_loss'])['test_loss']
        if len(test_loss_history) != 1:
            print(f'Warning: {id} has test_loss {test_loss_history}')
        config['test_loss'] = test_loss_history.item()
    except:
        config['test_loss'] = np.nan

    config_list.append(config)

results_df = pd.DataFrame(config_list)
results_df

0it [00:00, ?it/s]


""


In [41]:
results_df_new = pd.concat([results_df_old, results_df]).reset_index(drop=True)
results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))

# results_df.to_pickle(os.path.join(store_dir, 'results_df.pkl'))